#### yield特点
>- 只能用在函数内
>- 在函数内的任何地方出现yield关键字，哪怕永远无法被执行到，函数都会发生变异(执行该函数，不再直接执行)

In [1]:
def gen():
    print("hello")
    if False:
        yield

g = gen()
print(g)
print(type(gen))
print(type(g))

<generator object gen at 0x000001DE1EC79A80>
<class 'function'>
<class 'generator'>


In [2]:
import inspect

# 是函数，也是生成器函数
print(inspect.isfunction(gen))
print(inspect.isgeneratorfunction(gen))

# 生成器函数不是生成器
print(inspect.isgenerator(g))
print(inspect.isgenerator(gen))

True
True
True
False


In [3]:
# 生成器对象是迭代器

g = gen()
hasattr(g, '__iter__') and hasattr(g, '__next__')

g is iter(g) # 迭代器 要求 __iter__ 返回自身

True

#### yield的作用
>- 调用生成器函数不是直接执行其中的代码，而是返回一个对象；
>- 生成器函数内的代码，需要通过生成器对象来执行。

在作用上，类似于 class。

生成器对象就是迭代器，所以它的运行方式和迭代器是一致的。

In [4]:
def gen_42(meet_yield):
    print("Hola")
    if meet_yield:
        print("yield")
        yield 42
        print("yielded 42")
    print("bye")
    return "result"


In [5]:
g1 = gen_42(False)
x1 = next(g1) # return 语句触发 StopIteration
print(x1)

Hola
bye


StopIteration: result

In [6]:
g2 = gen_42(True)
x2 = next(g2)
print(x2)

Hola
yield
42


In [7]:
next(g2) # 从上一次 yield 继续执行，直到遇到下一个 yield 或 return

yielded 42
bye


StopIteration: result

In [8]:
def count(start = 0, step = 1):
    n = start
    while True:
        yield n
        n += step

#### 生成器的 4 个状态
>- 当调用生成器函数得到生成器对象时
>>- 此时的生成器对象可以理解为处于初始状态
>- 通过 next() 调用生成器对象，对应的生成器函数代码开始运行
>>- 此时生成器对象处于运行中状态
>- 如果遇到了 yield 语句，next() 返回时
>>- yield 语句右边的对象作为 next() 的返回值
>>- 生成器在 yield 语句所在的位置暂停，当再次使用 next() 时继续从该位置继续运行
>- 如果执行到函数结束，则抛出 StopIteration 异常
>>- 不管是使用了 return 语句显式地返回值，或者默认返回 None 值，返回值都只能作为异常的值一并抛出
>>- 此时的生成器对象处于结束的状态
>>- 对于已经结束的生成器对象再次调用 next()，直接抛出 StopIteration 异常，并且不含返回值

#### 应用场景

>- 定义一个容器类的可迭代对象，为该对象实现__iter__接口

In [9]:
# 场景一
# 迭代器类
class MyCustomDataIterator:
    def __init__(self, data):
        self.data = data
        self.index = -1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.index += 1
        if self.index < self.data.size:
            return self.data[self.index]
        else:
            return StopIteration

# 可迭代数据类
class MyCustomData:
    @property
    def size(self):
        return self.size
    
    def get_value(self, index):
        return index
    
    def __iter__(self):
        return MyCustomDataIterator(self)

In [10]:
# 可迭代数据类
class MyCustomData:
    @property
    def size(self):
        return self.size
    
    def get_value(self, index):
        return index
    
    def __iter__(self):
        index = -1
        while index < 2:
            index += 1
            yield self.get_value(index)
            return self.data[self.index]


##### 定义一个处理其他可迭代对象的迭代器

In [11]:
BLACK_LIST = ["kill", "murder"]

class SuzhiIterator:
    def __init__(self, actions):
        self.actions = actions
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        while self.index < len(self.actions):
            self.index += 1
            action = self.actions[self.index]
            if action in BLACK_LIST:
                continue
            elif action == "sleep":
                return "sleeping"
            else:
                return action
        raise StopIteration

In [12]:
BLACK_LIST = ["kill", "murder"]

def Suzhi(actions):
    for action in actions:
        if action in BLACK_LIST:
            continue
        elif action == "sleep":
            yield "sleeping"
        else:
            yield action

actions = ["exercise", "murder", "sleep"]
for x in Suzhi(actions):
    print(x)

exercise
sleeping


>- 定义一个不依赖数据存储的数据生成器

In [13]:
class DownCount:
    def __init__(self, start):
        self.start = start
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.start < 0:
            raise StopIteration
        else:
            current = self.start
            self.start -= 1
            return current

In [14]:
def DownCount(start):
    while start >= 0:
        yield start
        start -= 1

for x in DownCount(3):
    print(x)

3
2
1
0
